In [5]:
import pandas as pd
import joblib

# ---------------- Load model + expected cols ----------------
model = joblib.load("gb_vital_trend_model.joblib")
expected_cols = pd.read_csv("expected_columns.csv", header=None)[0].tolist()

# ---------------- Example input ----------------
sample = {
    "AT_START": 85.0,
    "After_6hr": 90.0,
    "After_12hr": 88.0,
    "After_18hr": 92.0,
    "DOSE_VAL_RX": 1000,
    "ROUTE": "IV",
    "Antibiotic_category": "Glycopeptides",
    # Antibiotics administered (others default to 0)
    "abx__Vancomycin": 1,
}

# ---------------- Prepare DataFrame ----------------
X_new = pd.DataFrame([sample])

# Add missing columns in one go
X_new = X_new.reindex(columns=expected_cols)

# Fill abx__* with 0, leave other NaNs for imputers
abx_cols = [c for c in expected_cols if c.startswith("abx__")]
X_new[abx_cols] = X_new[abx_cols].fillna(0)

# ---------------- Predict ----------------
pred = model.predict(X_new)[0]           # 0 = decrease, 1 = increase
prob = model.predict_proba(X_new)[0, 1]  # P(increase)

print("Prediction:", "Increase dosage" if pred == 1 else "Decrease dosage")
print("Confidence:", round(prob, 3))


Prediction: Increase dosage
Confidence: 0.736
